In [1]:
import sys
ana_helper_path = "/project/def-mdiamond/tomren/jupyter/Mu_helper/muhelper/"
sys.path.append(ana_helper_path)

In [2]:
#!/usr/bin/env python
# import analyzer
import importlib
from importlib import reload
import os, sys, glob, warnings, glob
import scipy
import numpy as np
import scipy as sp
import joblib
# from tqdm.notebook import tqdm
from tqdm import tqdm
import copy as cp

# ROOT
import ROOT as root

# Matplotlib
import matplotlib.pyplot as plt
from matplotlib import collections, colors, transforms

from pylab import *
%matplotlib inline
%config InlineBackend.figure_format='retina'
# %matplotlib widget

import pprint
pp = pprint.PrettyPrinter(indent=4)


# -------------------------
# All other scripits
import include_modules_root as rt
# Figure configuration are saved in this file include_figure_preset.py
from include_figure_preset import * 
# Set Figure font family, fontsize, ticks, etc.
plt_config(family="san-serif", fontsize_multi=1) # or "serif", or an exact font name


# Redefine a function to save figures with common settings 
fig_prefix = "plots/"    # It's good to keep figures in a separate folder. Can also be set to None.
fig_format = "jpg"      # for multiple formats, e.g.: "pdf,png"
SAVE_FIG = False         # Use this flag to turn the figure saving on or off, so that you don't need to modify all notebook to save figure.
# You can then do `savefig(filename_without_extension)` to save your plots with these settings
savefig = Save_fig(fig_prefix=fig_prefix, exts=fig_format, SAVE= SAVE_FIG, dpi=300)

Welcome to JupyROOT 6.24/06

RooFit v3.60 -- Developed by Wouter Verkerke and David Kirkby 
                Copyright (C) 2000-2013 NIKHEF, University of California & Stanford University
                All rights reserved, please read http://roofit.sourceforge.net/license.txt



In [3]:
# Figure configuration are saved in this file include_figure_preset.py
from include_figure_preset import * 
# Set Figure font family, fontsize, ticks, etc.
plt_config(family="san-serif", fontsize_multi=1) # or "serif", or an exact font name


# Redefine a function to save figures with common settings 
fig_prefix = "plots/"    # It's good to keep figures in a separate folder. Can also be set to None.
fig_format = "jpg"      # for multiple formats, e.g.: "pdf,png"
SAVE_FIG = False         # Use this flag to turn the figure saving on or off, so that you don't need to modify all notebook to save figure.
# You can then do `savefig(filename_without_extension)` to save your plots with these settings
savefig = Save_fig(fig_prefix=fig_prefix, exts=fig_format, SAVE= SAVE_FIG, dpi=300)

In [6]:
DATA_DIR    = "/project/rrg-mdiamond/tomren/mudata/background/"

## Universal names, Input file, output filenames, etc

In [ ]:
#--- Get a list of filenames to process ---
energy_list = [0.1, 0.2, 0.5, 1, 3, 10, 30, 100]
name_list = ["muon", "pion", "electron"]
name_list_latex = ["$\mu^-$", "$\pi^+$", "$e^-$"]
pdgid_list = [13, 211, 11]

INDS_PAR = [2,0,3,6,4,5] # from CMS coord to DET coord. 
PAR_LABELS=["$x_0$ [cm]","$y_0$ [cm]", "$t_0$ [ns]", "$v_x$ [cm/ns]", "$v_y$ [cm/ns]", "$v_z$ [cm/ns]"]
PAR_LABELS_CMS=["$x_0$ [cm]","$y_0$ [cm]","$z_0$ [cm]", "$t_0$ [ns]", "$v_x$ [cm/ns]", "$v_y$ [cm/ns]", "$v_z$ [cm/ns]"]
PAR_LABELS_RAW=["x0", "y0", "z0", "t0", "vx", "vy", "vz"]
PAR_PLOT_RANGES=np.array([[-15,15],[-15,15],[-4,4],[-2,2], [-2,2], [-6,6]])

In [12]:
file_list = {}

for name in name_list:
    file_list_temp = []
    for energy in energy_list:
        files=glob.glob(f"{DATA_DIR}/{name}_{energy}_GeV/*/*/stat_seedmod.root",)
        #files=util.Utils.sortByExt(files)
        if len(files)>=1:
            file_list_temp.append(files[0])
        if len(files)>1:
            print(f"More than one file for {name} at {energy} GeV")
    file_list[name] = file_list_temp

# Detector Geometry 

In [ ]:
gap = 100
module_height = 800
BoxLimits = [  [-5000.0, 5000.0],  [6000.0 + 547, 8917.0 + 547],  [6895.1, 17000.0]]
ModuleYLims=[[6547.0, 6547.0 + module_height], [6547.0 + module_height + gap, 6547.0 + 2*module_height + gap ], [6547.0 + 2*module_height + 2*gap,6547.0 + 3*module_height + 2*gap]] 
ModuleXLims = [ [-4950. + 1000.*n, -4050. + 1000*n] for n in range(10) ]
ModuleZLims = [ [7000.  + 1000.*n,  7900. + 1000*n] for n in range(10) ]
scintillator_height_all = 2.6 # 2cm +0.3*2Al case
wall_gap = 1.0
wall_gap2 = 100.0
wall_height = 2600.0
wall_start_y = ModuleYLims[0][0] - 3
z_min_wall = ModuleZLims[0][0] - wall_gap - scintillator_height_all
MODULE_Z_RANGE_CMS_cm = [70_00, 70_00+90_00]
MODULE_Y_RANGE_CMS_cm = [85_47, 85_47+11_00]
cm = 1
LayerYLims= [[6547.0,6547.0+scintillator_height_all],
            [6629.6,6629.6+scintillator_height_all],
            [8547.0,8547.0+scintillator_height_all],
            [8629.6,8629.6+scintillator_height_all],
            [9132.2,9132.2+scintillator_height_all],
            [9214.8,9214.8+scintillator_height_all],
            [9297.4,9297.4+scintillator_height_all],
            [9380.0,9380.0+scintillator_height_all],
            [9462.6,9462.6+scintillator_height_all],
            [9545.2,9545.2+scintillator_height_all]]
ymin=LayerYLims[0][0] - 10
ymax=LayerYLims[-1][1] + 10
floor_midpoint = [(6547.0 + 6547.0+scintillator_height_all)/2,(6629.6 + 6629.6+scintillator_height_all)/2]
wall_midpoint= [(ModuleZLims[0][0]- wall_gap -wall_gap2 - scintillator_height_all*1.5) ,(ModuleZLims[0][0] - wall_gap - scintillator_height_all/2.0)]
xLims=[BoxLimits[0][0],BoxLimits[0][1]]
yLims=[BoxLimits[1][0],BoxLimits[1][1]]
zLims=[BoxLimits[2][0],BoxLimits[2][1]]



bincenters=[0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50]
area_loss=[0.18302325581395348,
 0.19174418604651164,
 0.20046511627906977,
 0.2091860465116279,
 0.21790697674418605,
 0.22662790697674418,
 0.23534883720930233,
 0.24406976744186046,
 0.2527906976744186,
 0.26151162790697674,
 0.2702325581395349]







def inBox(hit):
    x,y,z=hit[:3]
    if x >= xLims[0] and x <= xLims[1]:
        if y >= yLims[0] and y <= yLims[1]:
            if z >= zLims[0] and z <= zLims[1]:
                return True
        return False


def inGap(hit):
    x,y,z = hit[:3]
    if z < 7000:
        for j in range(len(ModuleYLims)-1):
            if  ModuleYLims[j][1] < y < ModuleYLims[j+1][0]:
                return True    
        for k in range(len(ModuleXLims)-1): 
            if  ModuleXLims[k][1] < x < ModuleXLims[k+1][0]:
                return True
        return False
    elif y < 8547.0 and z >= 7000 :
        for i in range(len(ModuleXLims)-1):
            if  ModuleXLims[i][1] < x < ModuleXLims[i+1][0]:
                return True
            if  ModuleZLims[i][1] < z < ModuleZLims[i+1][0]:
                return True
        return False
    else:
        return False

    
def inModuleX(xVal):
    for moduleN, moduleLims in enumerate(ModuleXLims):
        if xVal > moduleLims[0] and xVal < moduleLims[1]:
            return moduleN

def inModuleZ(zVal):
    for moduleN, moduleLims in enumerate(ModuleZLims):
        if zVal > moduleLims[0] and zVal < moduleLims[1]:
            return moduleN
        
def inModuleY(yVal):
    for moduleN, moduleLims in enumerate(ModuleYLims):
        if yVal > moduleLims[0] and yVal < moduleLims[1]:
            return moduleN
def inFW(hit):
# returns the layer number given the y value 
# if hit is in the wall returns zero 
# non wall/floor hit inLayer(hit->y) > 2
    yVal= hit[1]
    zVal= hit[2]
    if zVal < 7000:
        return -1

    scintillator_height_all = 2.6
    LayerYLims= [[6547.0,6547.0+scintillator_height_all],
                 [6629.6,6629.6+scintillator_height_all],
                 [8547.0,8547.0+scintillator_height_all],
                 [8629.6,8629.6+scintillator_height_all],
                 [9132.2,9132.2+scintillator_height_all],
                 [9214.8,9214.8+scintillator_height_all],
                 [9297.4,9297.4+scintillator_height_all],
                 [9380.0,9380.0+scintillator_height_all],
                 [9462.6,9462.6+scintillator_height_all],
                 [9545.2,9545.2+scintillator_height_all]]
    for i in range(len(LayerYLims)):
        if yVal > LayerYLims[i][0] and yVal < LayerYLims[i][1]:
            return i+1

    return 0


def track_event_sort(data):
    event_pointer=[]
    for ind in data["event_ind"]:
        ind_list=[]
        for i in range(len(data["event_ind"])):
            if data["event_ind"][i]==ind:
                ind_list.append(i)
        event_pointer.append(ind_list)
    return event_pointer

# Vertex Fit

In [ ]:
def get_d2g_dt(fw,track_param,track):

    dist = 999
    dist_=[]
    
    #Projection to floor#
    dt = fw[-1]-track_param[-1]
    cental = track_point_propagation(track_param,dt)
    
    
    #projection distance from IP #
    track_new_loc = np.array( TK.floor_projection(0,track))
    dr = np.linalg.norm(track_new_loc[:3])
    
    
    # Total covariance is the track cov + hit cov
    #track_new_pcov_inv = np.linalg.inv(track_cov_propagation(track_param, track_cov, dt, dt_err_add=1)+ np.diag(hit_uncertainty**2))
    #chi2 = dr.T@track_new_pcov_inv@dr

    if inBox(cental) != True:
        return -999, dr 
    if inGap(cental):
        return 0 , dr 
    elif inFW(fw) ==-1:
        for i in range(2):
            try:
                xLim=ModuleXLims[inModuleX(cental[0])]
                yLim=ModuleYLims[inModuleY(cental[1])]
            except:
                continue
            if abs(cental[0]-xLim[i]) < dist :
                dist = abs(cental[0]-xLim[i]) 
            if abs(cental[1]-yLim[i]) < dist :
                dist = abs(cental[1]-yLim[i])  
        dist_.append(dist)
    elif -1 < inFW(fw) <= 2 :
        for i in range(2):
            try:
                xLim=ModuleXLims[inModuleX(cental[0])]
                zLim=ModuleZLims[inModuleZ(cental[2])]
            except:
                continue
            if abs(cental[0]-xLim[i]) < dist :
                dist = abs(cental[0]-xLim[i]) 
            if abs(cental[2]-zLim[i]) < dist :
                dist = abs(cental[2]-zLim[i])  
        dist_.append(dist)
    if len(dist_)==0:
        return -999, dr 
    if np.min(dist_) == 999:
        return -999, dr 
    return np.min(dist_), dr



def get_d2gap_dy(mid_points,track):
    
    
    dist_=[]
    dist=999
    for mid in midpoints:
        if inBox(cental) != True:
            return -999, dr 
        if inGap(cental):
            return 0 , dr 
        elif inFW(fw) ==-1:
            for i in range(2):
                try:
                    xLim=ModuleXLims[inModuleX(cental[0])]
                    yLim=ModuleYLims[inModuleY(cental[1])]
                except:
                    continue
                if abs(cental[0]-xLim[i]) < dist :
                    dist = abs(cental[0]-xLim[i]) 
                if abs(cental[1]-yLim[i]) < dist :
                    dist = abs(cental[1]-yLim[i])  
            dist_.append(dist)
        elif -1 < inFW(fw) <= 2 :
            for i in range(2):
                try:
                    xLim=ModuleXLims[inModuleX(cental[0])]
                    zLim=ModuleZLims[inModuleZ(cental[2])]
                except:
                    continue
                if abs(cental[0]-xLim[i]) < dist :
                    dist = abs(cental[0]-xLim[i]) 
                if abs(cental[2]-zLim[i]) < dist :
                    dist = abs(cental[2]-zLim[i])  
            dist_.append(dist)
    if len(dist_)==0:
        return -999, dr 
    if np.min(dist_) == 999:
        return -999, dr 
    return np.min(dist_), dr 

In [ ]:

# projcets the track Parameter & covariance using the time difference between the f/w digitized hits and the track#
# then calculates the chi2 #
def fwhits_to_chi2(hit,track_info):
    # Hit uncertainty:
    track_param =track_info[1]
    track_cov  =track_info[0]
    hit_uncertainty = get_hit_uncertainty(1, hit[1])

    # Method 1: Calculate chi2 between F/W hits and track
    dt = hit[-1]-track_param[-1]
    track_new_loc = np.array(track_point_propagation(track_param,dt))
    dr = track_new_loc[:3] - hit[:3]
    # Total covariance is the track cov + hit cov
    track_new_pcov_inv = np.linalg.inv(track_cov_propagation(track_param, track_cov, dt, dt_err_add=1)+ np.diag(hit_uncertainty**2))
    chi2 = dr.T@track_new_pcov_inv@dr
    return chi2 , np.linalg.norm(dr)


# calculates the chi2 for the IP tracklet veto#
def Track_to_IPtrack_chi2(hit, track_info):
    track_param =track_info[1]
    track_cov  =track_info[0]
    hit_uncertainty = get_hit_uncertainty(1, hit[1])
    
    # defining the IP tracklet #
    dr_IP = np.linalg.norm(hit[:3])
    dt_IP = dr_IP/29.94
    v0_IP = hit[:3]/dt_IP
    # IP tracklet parameters and initial covariance #
    track2_par = [hit[0], hit[1], hit[2], v0_IP[0], v0_IP[1], v0_IP[2], hit[3]]
    track2_cov = IP_track_cov(hit, np.append(hit_uncertainty,1))
    
    # minimizing the sum of chi2 distances of both the tracklet and reconstructed track to find a common vertex#
    fit1 = fit_vertex([track_param, track2_par], [track_cov, track2_cov], seedinedx=[0,1])
    
        # Calculate distance #
    vertex_par=np.array(list(fit1.values))
    vertex_par_detcoor = util.coord_cms2det(vertex_par[:3])
    hit_detcoor = util.coord_cms2det(hit[:3])
    track1_location_at_vertex = np.array(track_point_propagation(track_param,vertex_par[-1]-track_param[-1]))
    track2_location_at_vertex = np.array(track_point_propagation(track2_par, vertex_par[-1]-track2_par[-1]))
    dr_fit = np.linalg.norm(track1_location_at_vertex[:3] - track2_location_at_vertex[:3])
    dist_to_IPtrack= dr_fit
    dist2_to_IPtrack = find_distance_closestapproach(track_param, track2_par)
    
    # deflection 
    
    momentum_measured = track_param[3:6]
    momentum_measured_unit = momentum_measured/np.linalg.norm(momentum_measured)
    
    
    return fit1.fval , dist_to_IPtrack , dist2_to_IPtrack 

# function to calculate the initial tracklet covariance #
def IP_track_cov(hit, hit_unc):
    """
    Construct the covariance of an "IP track" connecting from IP to a given hit
    
    Two points:
    # IP: [0, 0, 0, t-dt], Hit: [x, y, z, t] 
    Track:
    # Track: [x0, y0, z0, vx, vy, vx, t0] = [x, y, z, x/dt, y/dt, z/dt, t], in which dt = dr/c = sqrt(x^2+y^2+z^2)/c    
    
    """
    x,y,z,t = hit[:4]
    c = 29.979
    dt= np.linalg.norm(hit[:3])/c
    dt3c2 = dt**3 * c**2
    
    # tracklet Jacobian #
    # dependent on the difinition of the tracklet #
    
    jac = np.array([[1,    0,     0,    1/dt - x**2/dt3c2,         - x*y/dt3c2,         - x*z/dt3c2,  0],
                   [0,    1,     0,         -  x*y/dt3c2,   1/dt - y**2/dt3c2,         - z*y/dt3c2,  0],
                   [0,    0,     1,         -  x*z/dt3c2,         - z*y/dt3c2,   1/dt - z**2/dt3c2,  0],
                   [0,    0,     0,                    0,                   0,                   0,  1]])

    track_cov = jac.T @ np.diag(hit_unc**2) @ jac
    return track_cov       




# Minute Minimization Step#
class chi2_vertex:
    def __init__(self, tracks, track_covs):
        self.tracks=tracks
        self.track_covs=track_covs
        self.func_code = iminuit.util.make_func_code(['x0', 'y0', 'z0', 't0'])
        self.errordef=1 # use least square
    def __call__(self, x0, y0, z0, t0):
        error=0
        for itrack in range(len(self.tracks)):
            error+= chi2_distance_to(self.tracks[itrack], self.track_covs[itrack], [x0, y0, z0], t0, point_t_err = 0)
        return error        

    
def fit_vertex(track_pars, track_covs, seedinedx=[0,1], ncall=2000):
    x0_init,y0_init, z0_init,t0_init = find_midpont_closestapproach(track_pars[seedinedx[0]], track_pars[seedinedx[1]])

    m = Minuit(chi2_vertex(track_pars, track_covs),x0=x0_init, y0=y0_init, z0=z0_init, t0=t0_init)
    limit_ext=2_00 # Extend the limit to beyond the decay volume
    m.limits["x0"]=(detector.Detector.BoxLimits[0][0]-limit_ext, detector.Detector.BoxLimits[0][1]+limit_ext)
    m.limits["y0"]=(detector.Detector.BoxLimits[1][0]-limit_ext, detector.Detector.BoxLimits[1][1]+limit_ext)
    m.limits["z0"]=(detector.Detector.BoxLimits[2][0]-limit_ext, detector.Detector.BoxLimits[2][1]+limit_ext)
    m.limits["t0"]=(0,1000)
    m.errors["x0"]=1
    m.errors["y0"]=1
    m.errors["z0"]=1
    m.errors["t0"]=1

    m.migrad(ncall=ncall)  # run optimiser
    # m.hesse()   # run covariance estimator
    
    return m

In [ ]:
def get_deflection(track_par,track):
    x,y,z=track[:3] 
    utrack=np.linalg.norm([x[-1] - x[0], y[-1] - y[0], z[-1] - z[0]])
    par_ip=track_par[:3]
    upar=np.linalg.norm(track_par[:3])
    angle=np.dot([x[-1] - x[0], y[-1] - y[0], z[-1] - z[0]],track_par[:3])/(utrack*upar)
    return np.arccos(angle)/np.pi*180

In [ ]:
# Distance to Gap intercept#

def inFloor_Wall(point):
    zval=point[2]
    yval=point[1]
    
    if zval > 6895.1 and zval < 7000:
        return True
    
    elif yval > 6548 and yval <= 6630 +scintillator_height_all and zval >= 7000 :
        return True
    return False

#Extends the reconstructed track until it intercepts either the floor/wall#
def find_d2g_intercept(track_par,track):
    if track_par[-1] > 400:
        t= track_par[-1] - 399
    else:
        t = 50
    point=track_par[:3]
    found = False
    while found == False:
        t= t + 3
        dt= - t
        point = track_point_propagation(track_par,dt)
        # if it intercepts with the floor, run track projection to floor midpoints#
        if point[1] # if it intercepts with the floor, run track projection to floor midpoints#> 6548 and point[1] <= 6630:
            return d2gap(track, "floor")
        # if it intercepts with the wall, run track projection to wall midpoints#
        elif point[2] > 6895.1 and point[2] < 7000:
            return d2gap(track, "wall")

# calculates the distance to gap by finding the projection point to either the modules of the floor/wall, then calculating the distance to the nearest edge.#    
def d2gap(track,dirc):
    dist_list=[]
    dist = 99999
    dist_=[]
    central_point=[]    
    if dirc=="wall":
        for zval in wall_midpoint:
            central_point.append(TK.wall_projection(zval,track))
    elif  dirc=="floor":
        for yval in floor_midpoint:
            central_point.append(TK.floor_projection(yval,track))
    for cental in (central_point):
        if inBox(cental) != True:
            dist_.append(999)
    
        elif inGap(cental):
            return  0
        elif dirc=="wall":
            for i in range(2):
                try:
                    xLim=ModuleXLims[inModuleX(cental[0])]
                    yLim=ModuleYLims[inModuleY(cental[1])]
                except:
                    continue
                if abs(cental[0]-xLim[i]) < dist :
                    dist = abs(cental[0]-xLim[i]) 
                if abs(cental[1]-yLim[i]) < dist :
                    dist = abs(cental[1]-yLim[i])  
            dist_.append(dist)
        elif dirc=="floor":
            for i in range(2):
                try:
                    xLim=ModuleXLims[inModuleX(cental[0])]
                    zLim=ModuleZLims[inModuleZ(cental[2])]
                except:
                    continue
                if abs(cental[0]-xLim[i]) < dist :
                    dist = abs(cental[0]-xLim[i]) 
                if abs(cental[2]-zLim[i]) < dist :
                    dist = abs(cental[2]-zLim[i])  
            dist_.append(dist)
        if len(dist_)==0:
            continue
        dist_list.append(min(dist_))
    if len(dist_list)==0:
        return -999
    if np.min(dist_list) == 999:
        return -999
    return np.min(dist_list)
        

# Useful functions

In [1]:
def plot_res_pull(data, figs=None, make_legend=False, plot_gauss1=True, label="", figsize=(10,3.5)):
    axlabels=["$x_0$ [cm]","$y_0$ [cm]", "$t_0$ [ns]", "$v_x$ [cm/ns]", "$v_y$ [cm/ns]", "$v_z$ [cm/ns]"]
    ranges=np.array([[-15,15],[-15,15],[-4,4],[-2,2], [-2,2], [-6,6]])

    mask_recon_success=data["mask_recon_success"]
    recon     =np.array(data["recon"])[mask_recon_success]
    recon_unc =np.array(data["recon_error"])[mask_recon_success]
    truth     =np.array(data["truth"])[mask_recon_success]

    
    if figs is None:
        figs = []
        for i in range(6):
            fig,axs=plt.subplots(1,2,figsize=figsize)
            figs.append(fig)

    for ipar in range(6):
        ipar_cms = INDS_PAR[ipar]
        residual=(recon-truth)[:,ipar_cms]
        pull=util.pull(residual,0,recon_unc[:,ipar_cms])

        # plotrange=[-2*np.std(residual_kf), 2*np.std(residual_kf)]
        plotrange=ranges[ipar]

        # fig,axs=plt.subplots(1,2,figsize=(10,4))
        figure(figs[ipar])
        axs=figs[ipar].axes
        plt.sca(axs[0])
        n,ibins,p = hist(residual,range=plotrange,histtype="step",label=label,bins=100);
        yscale("log")
        ymin, ymax = gca().get_ylim()
        ylim(bottom=1, top = max([max(n)*2,ymax]))        
        xlabel("Reco-truth, "+axlabels[ipar])
        ylabel("[counts/bin]")

        plt.sca(axs[1])
        n,ibins,p = hist(pull,range=[-5,5],histtype="step",label=label,bins=100);

        bincenters=0.5*(ibins[1:]+ibins[:-1])
        y = util.Utils.Gauss(bincenters, max(n),0,1)
        if plot_gauss1:
            plt.plot(bincenters,y,color="r",label=r"Gauss, $\sigma$=1",linestyle=":")
            
        yscale("log")
        ymin, ymax = gca().get_ylim()
        ylim(bottom=1, top = max([max(n)*2,ymax]))
        xlabel(r"$\frac{Reco-truth}{Unc_{reco}}$, "+axlabels[ipar].split(" ")[0]+" [$\sigma$]")
        
        if make_legend:
            plt.legend(loc=(1.01,0),fontsize=11)
    
    return figs

In [16]:
def calc_eff(res, PDG_TRUTH = 13):
    recon     =np.array(res["recon"])
    recon_unc =np.array(res["recon_error"])
    truth     =np.array(res["truth"])
    
    
    mask_recon_success=res["mask_recon_success"]
    mask_recon_able = (res["truth_nlayer"]>=5)& (res["truth_nlayer"]<=9) # layer 2 is the bottom layer, 9 is the top
    
    mask_identified= np.zeros(len(res["recon"]),dtype=bool)
    for i in range(len(mask_identified)):
        n_truth_id = sum(np.array(res["par_km_pdgids"][i])==PDG_TRUTH)
        n_false_id = sum(np.array(res["par_km_pdgids"][i])!=PDG_TRUTH)
        track_purity = n_truth_id/(n_truth_id+n_false_id)
        if n_truth_id>=4 and n_false_id==0:
            mask_identified[i]=True
    
    n_events = len(mask_recon_success)
    n_success = np.sum(mask_recon_success)
    
        
    # Make a fixed range cut for tight and looser track
    diffx = recon[:,2]-truth[:,2]
    diffy = recon[:,0]-truth[:,0]
    diffvx = recon[:,6]-truth[:,6]
    diffvy = recon[:,4]-truth[:,4] 
    mask_TIGHT  = (np.abs(diffx)<5) & (np.abs(diffy)<5) & (np.abs(diffvx)<0.5) & (np.abs(diffvy)<0.5)
    mask_LOOSER = (np.abs(diffx)<10) & (np.abs(diffy)<10) & (np.abs(diffvx)<1) & (np.abs(diffvy)<1)
            
            
    eff_raw=util.Utils.flatten1d(list(rt.BayesDivide([sum(mask_recon_success)],[len(mask_recon_success)])))
    eff_abs_tight=util.Utils.flatten1d(list(rt.BayesDivide([sum(mask_TIGHT&mask_recon_success)],[len(mask_recon_success)])))
    eff_abs_loose=util.Utils.flatten1d(list(rt.BayesDivide([sum(mask_LOOSER&mask_recon_success)],[len(mask_recon_success)])))

    eff_reconstructible = util.Utils.flatten1d(list(rt.BayesDivide([sum(mask_recon_able)],[len(mask_recon_success)])))
    eff_identified=util.Utils.flatten1d(list(rt.BayesDivide([sum(mask_identified&mask_recon_able)],[sum(mask_recon_able)])))
    eff_resolution =util.Utils.flatten1d(list(rt.BayesDivide([sum(mask_LOOSER&mask_identified&mask_recon_able)],[sum(mask_identified&mask_recon_able)])))
    
    if len(eff_resolution)==0:
        eff_resolution = [0,0,0]
    
    return eff_raw, eff_abs_tight, eff_abs_loose, eff_reconstructible,  eff_identified, eff_resolution


def calc_resolution(res):
    recon     =np.array(res["recon"])
    recon_unc =np.array(res["recon_error"])
    truth     =np.array(res["truth"])
    
    
    mask_recon_success=res["mask_recon_success"]
    ranges=np.array([[-15,15],[-15,15],[-4,4],[-2,2], [-2,2], [-6,6]])
    ind = INDS_PAR
    
    sigmas = []
    sigmas_unc = []
    fwhms = []
    for i in range(6):
        diff = recon[:,ind[i]]-truth[:,ind[i]]
        n,ibins= np.histogram(diff,bins=100,range=ranges[i]);
        
        
        bincenters=0.5*(ibins[1:]+ibins[:-1])
        yerr=np.sqrt(n);yerr[yerr==0]=1
        
        popt,pcov = rt.fit_tg(bincenters,n,yerr=yerr,function="gaus")
        perr = np.sqrt(np.diag(pcov))
        
        fw = util.Utils.fwhm(bincenters, n)
        fwhm = fw[1]-fw[0]
        
        sigmas.append(popt[2])
        sigmas_unc.append(perr[2])
        fwhms.append(fwhm)
        
    return sigmas,sigmas_unc,fwhms
    

In [ ]:
def get_km_alltracks(filename, results_fit = None, tree_name="integral_tree", truth_pids = [13,13], nevents=-1):
    """
    Parameters
    ---
    nevents: 
      -1: all events
      (start, stop) from start to stop
    """
    
    if results_fit is None:
        results_fit={}

        
    results_fit["Entry"]=[]               # ROOT event entry number
    results_fit["ndigi"]=[]               # Total number of digitized hits
    results_fit["Digi_track_id"]=[]
    results_fit["mask_recon_success_track"]=[]    # Boolean mask 
    results_fit["mask_recon_success_vertex"]=[]   # Boolean mask 
    results_fit["mask_reconstructible_vertex"]=[]   # Boolean mask, if an event has two tracks with 4+ hits. The track is required to be from the direct input
    results_fit["mask_reconstructible2_vertex"]=[]   # Boolean mask, if an event has two tracks with 4+ hits. The track can be from any particle.  

    
    results_fit["tracks_truth"]=[]
    results_fit["tracks_truth_n"]=[]
    results_fit["tracks_truth_nlayer"]=[]
    results_fit["tracks_truth_pdgids"]=[]
    
    results_fit["tracks_recon"]=[]
    results_fit["tracks_recon_n"]=[]
    results_fit["tracks_recon_error"]=[]          # KF parameter uncertainty. Already taken sqrt()
    results_fit["tracks_ndigi"]=[]          #  number of digitized hits in the track
    results_fit["tracks_ndigi_false"]=[]    #  number of digitized hits in the track that are not from the truth
    results_fit["tracks_purity"]=[]         # List of PDG id of hits in this track 
    results_fit["tracks_pdgids"]=[]         # List of PDG id of hits in this track 
    results_fit["tracks_chi2"]=[]           # KF fit chi2
    
    results_fit["vertices_truth"]=[]
    results_fit["vertices_ntrack"]=[]
    results_fit["vertices_ntrack_truth"]=[]
    results_fit["vertices_recon"]=[]
    results_fit["vertices_recon_n"]=[]
    results_fit["vertices_recon_error"]=[]
    results_fit["vertices_recon_cov"]=[]
    results_fit["vertices_chi2"]=[]
    


    ev = event.Event(filename, 0, tree_name=tree_name)
    Tree=ev.Tree
    nevents_total = int(ev.Tree.GetEntries())
    cut=cutflow.sample_space("")
    
    if nevents==-1:
        nevents = [0, nevents_total]
    elif type(nevents) is int:
        if nevents_total<nevents:
            print(f"Requested events exceed total {nevents_total}")
        nevents = [0, min(nevents,nevents_total)]
    else:
        if nevents_total<nevents[1]:
            print(f"Requested events exceed total {nevents_total}")        
        nevents = [nevents[0], min(nevents[1],nevents_total)]

    for i_event in tqdm(range(nevents[0], nevents[1])):
        ev.EventNumber=i_event
        ev.Tree.GetEntry(i_event)
        ev.ExtractTruthPhysics_list()
        
        par_km_ndigi = ev.Tree.Digi_x.size()
        
        results_fit["Entry"].append(i_event)
        results_fit["ndigi"].append(par_km_ndigi)
        results_fit["Digi_track_id"].append(util.c2list(ev.Tree.Digi_track_id))
        ids=np.array(results_fit["Digi_track_id"][-1])
        ys = np.array(util.c2list(ev.Tree.Digi_y))
        n_reconstructible=0
        n_reconstructible2=0
        for g4id in range(0,200):
            if len(np.unique(ys[ids==g4id]))>=4:
                n_reconstructible+=1
        for g4id in np.unique(ids):
            if len(np.unique(ys[ids==g4id]))>=4:
                n_reconstructible2+=1                
        results_fit["vertices_ntrack_truth"].append(n_reconstructible)
        if n_reconstructible>=2:
            results_fit["mask_reconstructible_vertex"].append(True)
        else:
            results_fit["mask_reconstructible_vertex"].append(False)   
        if n_reconstructible2>=2:
            results_fit["mask_reconstructible2_vertex"].append(True)
        else:
            results_fit["mask_reconstructible2_vertex"].append(False)               

        # Get truth (speed need to be calculated by hand)
        try:
            # Truth position and speed
            TruthTracks = []
            Truths = []
            Truth_nlayer = []
            Truth_pdgids = []
            for track in ev.truthTrackList_list:
                if track[4][0] in truth_pids:
                    TruthTracks.append(track)
                    dt=track[3][1]-track[3][0]
                    vx=(track[0][1]-track[0][0])/dt
                    vy=(track[1][1]-track[1][0])/dt
                    vz=(track[2][1]-track[2][0])/dt
                    truth = [track[0][0], track[1][0], track[2][0], track[3][0],vx,vy,vz]  
                    Truths.append(truth)
                    Truth_nlayer.append(np.abs(cut.in_layer(track[1][-1])-cut.in_layer(track[1][0])))
                    Truth_pdgids.append(track[4][0])
            
            n_truthtracks=len(TruthTracks)
            results_fit["tracks_truth"].append(Truths)  
            results_fit["tracks_truth_n"].append(n_truthtracks)  
            results_fit["tracks_truth_nlayer"].append(Truth_nlayer)  
            
        except:
            n_truthtracks=0
            results_fit["tracks_truth"].append([[-9999]])
            results_fit["tracks_truth_n"].append(n_truthtracks)  
            results_fit["tracks_truth_nlayer"].append([[-9999]])
                  
        
        # If there is reconstruction:
        if len(ev.Tree.Track_k_m_z0)==0:
            tracks_recon = [[-9990, -9990, -9990, -9990, -9990, -9990, -9990]]
            tracks_recon_n = 0
            tracks_recon_error = [[-9990, -9990, -9990, -9990, -9990, -9990, -9990]]
            tracks_chi2 = [0]
            
            tracks_ndigi = [0]
            tracks_ndigi_false = [0]
            tracks_purity = [-999]
            tracks_pdgids = [0]
            results_fit["mask_recon_success_track"].append(False)
            
        else:
            tracks_recon = []
            tracks_recon_n = Tree.Track_k_m_z0.size()
            tracks_recon_error = []
            tracks_chi2 = []
            tracks_ndigi = []
            tracks_ndigi_false = []
            tracks_purity = []
            tracks_pdgids = []            
            
            
            # Select the reconstruction that is closest to truth
            track_digi_hit_inds = util.unzip(Tree.Track_k_m_hitIndices)
            track_truth_ids = util.unzip(Tree.Track_k_m_ids)
            track_digi_hit_len = np.array([len(i) for i in track_digi_hit_inds])
            if tracks_recon_n<n_truthtracks:
                results_fit["mask_recon_success_track"].append(False)
            else:
                results_fit["mask_recon_success_track"].append(True)                
                
            for i_track in range(len(TruthTracks)):
                truth = Truths[i_track]

                track_chi2s = []
                if len(track_digi_hit_inds)>1:
                    for track_ind in range(len(track_digi_hit_inds)):
                        recon_i = [Tree.Track_k_m_x0.at(track_ind), Tree.Track_k_m_y0.at(track_ind), Tree.Track_k_m_z0.at(track_ind), Tree.Track_k_m_t0.at(track_ind),Tree.Track_k_m_velX.at(track_ind), Tree.Track_k_m_velY.at(track_ind), Tree.Track_k_m_velZ.at(track_ind)]
                        recon_i_unc = [Tree.Track_k_m_ErrorX0.at(track_ind), Tree.Track_k_m_ErrorY0.at(track_ind), Tree.Track_k_m_ErrorZ0.at(track_ind), Tree.Track_k_m_ErrorT0.at(track_ind),Tree.Track_k_m_ErrorVx.at(track_ind), Tree.Track_k_m_ErrorVy.at(track_ind), Tree.Track_k_m_ErrorVz.at(track_ind)]
                        chi2 = util.chi2_calc(recon_i,truth,recon_i_unc)
                        track_chi2s.append(chi2)
                    track_ind = int(np.argmin(track_chi2s))
                else:
                    track_ind=0

                tracks_recon.append([ev.Tree.Track_k_m_x0.at(track_ind), ev.Tree.Track_k_m_y0.at(track_ind), ev.Tree.Track_k_m_z0.at(track_ind), ev.Tree.Track_k_m_t0.at(track_ind), ev.Tree.Track_k_m_velX.at(track_ind), ev.Tree.Track_k_m_velY.at(track_ind), ev.Tree.Track_k_m_velZ.at(track_ind)])
                tracks_recon_error.append([ev.Tree.Track_k_m_ErrorX0.at(track_ind), ev.Tree.Track_k_m_ErrorY0.at(track_ind), ev.Tree.Track_k_m_ErrorZ0.at(track_ind), ev.Tree.Track_k_m_ErrorT0.at(track_ind), ev.Tree.Track_k_m_ErrorVx.at(track_ind), ev.Tree.Track_k_m_ErrorVy.at(track_ind), ev.Tree.Track_k_m_ErrorVz.at(track_ind)])
                tracks_chi2.append(ev.Tree.Track_k_m_smooth_chi_sum.at(track_ind))
                
                track_hits_inds=track_digi_hit_inds[track_ind]
                truth_pid = TruthTracks[i_track][4][0]
                truth_track_id = TruthTracks[i_track][6][0]
                kalmantrack_truthtrack_ids = track_truth_ids[track_ind]
                
                tracks_pdgids.append([ev.Tree.Digi_pdg_id.at(i) for i in track_hits_inds])
                tracks_ndigi.append(len(track_hits_inds))
                tracks_ndigi_false.append(sum(np.array(kalmantrack_truthtrack_ids)!=truth_track_id))
                tracks_purity.append(1-tracks_ndigi_false[-1]/tracks_ndigi[-1])
                
        # Vertex========================================================
        if len(ev.Tree.Vertex_k_m_x)==0:
            vertex_recon = [[0]]
            vertex_recon_n=0
            vertex_recon_error=[[0]]
            vertices_chi2=[[0]]
            vertex_recon_cov=[]
            results_fit["mask_recon_success_vertex"].append(False) 
        else:
            results_fit["mask_recon_success_vertex"].append(True) 
            vertex_recon=[]
            vertex_recon_n = Tree.Vertex_k_m_x.size()
            vertex_recon_error=[]
            vertex_recon_cov=[]
            vertices_chi2=[]
            for iv in range(vertex_recon_n):
                v=[Tree.Vertex_k_m_x.at(iv),Tree.Vertex_k_m_y.at(iv),Tree.Vertex_k_m_z.at(iv),Tree.Vertex_k_m_t.at(iv)]
                vertex_recon.append(v)
                verr=[Tree.Vertex_k_m_ErrorX.at(iv),Tree.Vertex_k_m_ErrorY.at(iv),Tree.Vertex_k_m_ErrorZ.at(iv),Tree.Vertex_k_m_ErrorT.at(iv)]
                vertex_recon_error.append(verr)  
                vcov=[[Tree.Vertex_k_m_ErrorX.at(iv)**2, Tree.Vertex_k_m_cov_x_y.at(iv), Tree.Vertex_k_m_cov_x_z.at(iv), Tree.Vertex_k_m_cov_t_x.at(iv)],
                      [Tree.Vertex_k_m_cov_x_y.at(iv),   Tree.Vertex_k_m_ErrorY.at(iv)**2, Tree.Vertex_k_m_cov_y_z.at(iv), Tree.Vertex_k_m_cov_t_y.at(iv)],
                      [Tree.Vertex_k_m_cov_x_z.at(iv),   Tree.Vertex_k_m_cov_y_z.at(iv), Tree.Vertex_k_m_ErrorZ.at(iv)**2, Tree.Vertex_k_m_cov_t_z.at(iv)],
                      [Tree.Vertex_k_m_cov_t_x.at(iv),   Tree.Vertex_k_m_cov_t_y.at(iv), Tree.Vertex_k_m_cov_t_z.at(iv), Tree.Vertex_k_m_ErrorT.at(iv)**2]]
                vertex_recon_cov.append(vcov)
                vertices_chi2.append(Tree.Vertex_k_m_chi2.at(iv))
                
        vertex_truth_cms = [Tree.GenParticle_y.at(1)*0.1, -Tree.GenParticle_z.at(1)*0.1 + 8547, Tree.GenParticle_x.at(1)*0.1]
        vertex_track_inds = util.unzip(Tree.Vertex_k_m_trackIndices)
        vertex_ntrack = [len(ii) for ii in vertex_track_inds]
        
        
        results_fit["tracks_recon"].append(tracks_recon)
        results_fit["tracks_recon_n"].append(tracks_recon_n)
        results_fit["tracks_recon_error"].append(np.sqrt(tracks_recon_error))
        results_fit["tracks_chi2"].append(tracks_chi2)
        results_fit["tracks_ndigi"].append(tracks_ndigi)
        results_fit["tracks_ndigi_false"].append(tracks_ndigi_false)
        results_fit["tracks_purity"].append(tracks_purity)
        results_fit["tracks_pdgids"].append(tracks_pdgids)
        
        results_fit["vertices_truth"].append(vertex_truth_cms)
        results_fit["vertices_ntrack"].append(vertex_ntrack)
        results_fit["vertices_recon"].append(vertex_recon)
        results_fit["vertices_recon_n"].append(vertex_recon_n)
        results_fit["vertices_recon_error"].append(vertex_recon_error)
        results_fit["vertices_recon_cov"].append(vertex_recon_cov)
        results_fit["vertices_chi2"].append(vertices_chi2)
        
        
    for key in ["tracks_truth_n","tracks_recon_n","mask_recon_success_track", "mask_recon_success_vertex","vertices_recon_n", "vertices_ntrack_truth"]:
        results_fit[key]=np.array(results_fit[key])
        
        
    return results_fit


def closest_approach_midpoint(tr1, tr2):
    """
    Input:
    ---
    tr1,tr2:
        lists of track paramters [x0, y0, x0, vx, vy, vz, t0]
        
    return:
    ---
    midpoint([x,y,z]), midpoint_time, distance
    """

    rel_v = tr2[3:6] - tr1[3:6];
    rel_v2 = np.dot(rel_v, rel_v) 

    displacement = tr2[:3] - tr1[:3]; # position difference
    t_ca = -(  np.dot(displacement, rel_v) - np.dot((tr2[3:6]*tr2[6] - tr1[3:6]*tr1[6]), rel_v)  )/rel_v2;
    
    displacement = tr1[:3] - tr2[:3]; # position difference
    t_ca = (  np.dot(displacement, rel_v) + np.dot((tr2[3:6]*tr2[6] - tr1[3:6]*tr1[6]), rel_v)  )/rel_v2;    
    

    pos1 = tr1[:3] + tr1[3:6]*(t_ca - tr1[6]);
    pos2 = tr2[:3] + tr2[3:6]*(t_ca - tr2[6]);
    
    line_distance = np.linalg.norm((pos1- pos2))

    return (pos1 + pos2)*(0.5), t_ca, line_distance


def find_midpont_closestapproach(track1_par, track2_par):
    # Find the midpoint of two tracks with "closeset approach"
    # Test: find_midpont_closestapproach([0,0,0, 0,1,0 ,0], [0,1,-1, 0,0,1, 0])
    # These two lines should intersect at [0,1,0,1]
    r1 = np.array(track1_par[:3]); v1 = np.array(track1_par[3:6]); t1 = track1_par[-1]
    r2 = np.array(track2_par[:3]); v2 = np.array(track2_par[3:6]); t2 = track2_par[-1]
    rel_v = v2-v1
    rel_vsq = np.dot(rel_v, rel_v)
    displacement = r1-r2
    
    t_ca = ((displacement@rel_v.T) - ( v1*t1 - v2*t2)@rel_v.T)  /rel_vsq;
    pos1 = r1 + v1*(t_ca - t1);
    pos2 = r2 + v2*(t_ca - t2);
    
    mid =  (pos1 + pos2)*0.5;
    return [*mid, t_ca]

def find_distance_closestapproach(track1_par, track2_par):
    r1 = np.array(track1_par[:3]); v1 = np.array(track1_par[3:6]); t1 = track1_par[-1]
    r2 = np.array(track2_par[:3]); v2 = np.array(track2_par[3:6]); t2 = track2_par[-1]
    rel_v = v2-v1
    rel_vsq = np.dot(rel_v, rel_v)
    displacement = r1-r2
    
    t_ca = ((displacement@rel_v.T) - ( v1*t1 - v2*t2)@rel_v.T)  /rel_vsq;
    pos1 = r1 + v1*(t_ca - t1);
    pos2 = r2 + v2*(t_ca - t2);
    
    dist =  np.linalg.norm(pos1-pos2);
    return dist


def line_dist(tr1,tr2,t_ca):
    pos1 = tr1[:3] + tr1[3:6]*(t_ca - tr1[6]);
    pos2 = tr2[:3] + tr2[3:6]*(t_ca - tr2[6]);  
    displacement = pos1-pos2
    
    return np.dot(displacement,displacement)


def get_track_cov(Tree, i):
    i=int(i)
    return np.array([[Tree.Track_k_m_ErrorX0.at(i), 0                           , Tree.Track_k_m_cov_x_z.at(i), Tree.Track_k_m_cov_x_vx.at(i), Tree.Track_k_m_cov_x_vy.at(i), Tree.Track_k_m_cov_x_vz.at(i), Tree.Track_k_m_cov_x_t.at(i)],
                    [0                            , Tree.Track_k_m_ErrorY0.at(i), 0                           , 0,0,0,0],
                    [Tree.Track_k_m_cov_x_z.at(i), 0                            , Tree.Track_k_m_ErrorZ0.at(i), Tree.Track_k_m_cov_z_vx.at(i), Tree.Track_k_m_cov_z_vy.at(i), Tree.Track_k_m_cov_z_vz.at(i), Tree.Track_k_m_cov_t_z.at(i)],
                    [Tree.Track_k_m_cov_x_vx.at(i), 0                            , Tree.Track_k_m_cov_z_vx.at(i), Tree.Track_k_m_ErrorVx.at(i), Tree.Track_k_m_cov_vx_vy.at(i), Tree.Track_k_m_cov_vx_vz.at(i), Tree.Track_k_m_cov_t_vx.at(i)],
                    [Tree.Track_k_m_cov_x_vy.at(i), 0                            , Tree.Track_k_m_cov_z_vy.at(i), Tree.Track_k_m_cov_vx_vy.at(i), Tree.Track_k_m_ErrorVy.at(i), Tree.Track_k_m_cov_vy_vz.at(i), Tree.Track_k_m_cov_t_vy.at(i)],
                    [Tree.Track_k_m_cov_x_vz.at(i), 0                            , Tree.Track_k_m_cov_z_vz.at(i), Tree.Track_k_m_cov_vx_vz.at(i), Tree.Track_k_m_cov_vy_vz.at(i), Tree.Track_k_m_ErrorVz.at(i), Tree.Track_k_m_cov_t_vz.at(i)],
                    [Tree.Track_k_m_cov_x_t.at(i), 0                            , Tree.Track_k_m_cov_t_z.at(i), Tree.Track_k_m_cov_t_vx.at(i), Tree.Track_k_m_cov_t_vy.at(i), Tree.Track_k_m_cov_t_vz.at(i), Tree.Track_k_m_ErrorT0.at(i)],])

def get_track_param(Tree,track_ind):
    track_ind=int(track_ind)
    return np.array([Tree.Track_k_m_x0.at(track_ind), Tree.Track_k_m_y0.at(track_ind), Tree.Track_k_m_z0.at(track_ind),Tree.Track_k_m_velX.at(track_ind), Tree.Track_k_m_velY.at(track_ind), Tree.Track_k_m_velZ.at(track_ind), Tree.Track_k_m_t0.at(track_ind)])
def get_track(Tree,track_ind):
    track=[]
    track_ind=int(track_ind)
    track_digi_hit_inds = util.unzip(Tree.Track_k_m_hitIndices)
    x_s = util.unzip(Tree.x_estimates_m)
    y_s = util.unzip(Tree.y_estimates_m)
    z_s = util.unzip(Tree.z_estimates_m)
    for n in range(len(track_digi_hit_inds)):
        x,y,z,t=[],[],[],[]

    #fill recon information
        track.append([x_s[int(n)], y_s[int(n)], z_s[int(n)]])
    return [x_s[int(n)], y_s[int(n)], z_s[int(n)]]

def get_track_param_truth_p(Tree,track_ind):
    track_ind=int(track_ind)
    g4trackind = np.array(util.c2list(Tree.Hit_G4TrackId))
    hit_index = int(np.argmax(g4trackind==track_ind))
    
    return np.array([Tree.Hit_x.at(hit_index), Tree.Hit_y.at(hit_index), Tree.Hit_z.at(hit_index),Tree.Hit_particlePx.at(hit_index), Tree.Hit_particlePy.at(hit_index), Tree.Hit_particlePz.at(hit_index), Tree.Hit_time.at(hit_index)])

def get_track_param_truth_v(Tree,track_ind):
    track_ind=int(track_ind)
    g4trackind = np.array(util.c2list(Tree.Hit_G4TrackId))
    hit_index = int(np.argmax(g4trackind==track_ind))
    dx = Tree.Hit_x.at(hit_index+1)-Tree.Hit_x.at(hit_index)
    dy = Tree.Hit_y.at(hit_index+1)-Tree.Hit_y.at(hit_index)
    dz = Tree.Hit_z.at(hit_index+1)-Tree.Hit_z.at(hit_index)
    dt = Tree.Hit_time.at(hit_index+1)-Tree.Hit_time.at(hit_index)
    
    return np.array([Tree.Hit_x.at(hit_index), Tree.Hit_y.at(hit_index), Tree.Hit_z.at(hit_index), dx/dt, dy/dt, dz/dt, Tree.Hit_time.at(hit_index)])


def track_cov_propagation(track_popt, track_pcov, dt, dt_err_add=0):
    x0,y0,z0,vx,vy,vz,t0 = track_popt
    
    jac=np.array([[1,   0,   0,   dt,  0,   0,   -vx],
                  [0,   1,   0,    0, dt,   0,   -vy],
                  [0,   0,   1,    0,  0,  dt,   -vz]])
    
    # Times the Jacobian, and add uncertainty from the time
    point_pcov = jac@track_pcov@jac.T + np.diag([(vx*dt_err_add)**2, (vy*dt_err_add)**2, (vz*dt_err_add)**2])
    # point_pcov_inv=  np.linalg.inv(point_pcov)+ np.diag([1/(vx*dt_err_add)**2, 1/(vy*dt_err_add)**2, 1/(vz*dt_err_add)**2])
    
    return point_pcov

def get_hit_uncertainty(hit, digi_layer_id=None):
    hit_uncertainty = np.array(detector.Layer().uncertainty(digi_layer_id))
    if digi_layer_id<10:
        hit_uncertainty=np.array(hit_uncertainty)*100 # turn to cm, in CMS coordinate
    else:
        hit_uncertainty=np.array([hit_uncertainty[0]*100, hit_uncertainty[2]*100, hit_uncertainty[1]*100])

    return hit_uncertainty

def track_point_propagation(track_popt,dt):
    x0,y0,z0,vx,vy,vz,t0 = track_popt
    return [x0+vx*dt,y0+vy*dt,z0+vz*dt, t0+dt]

def chi2_distance_to(track_param, track_cov, point_xyz, point_t, point_t_err = 0):
    """
    track_param: (x0,y0,z0, vx,vy,vz, t)
    """
    dt = point_t - track_param[6]
    point_location = track_point_propagation(track_param, dt)
    
    residual_vector = point_xyz[:3] - np.array(point_location[:3])
    
    # point_pcov_inv = track_cov_propagation(track_param, track_cov, dt, dt_err_add=point_t_err)
    # chi2 = residual_vector.T @ point_pcov_inv @ residual_vector;
    
    point_pcov = track_cov_propagation(track_param, track_cov, dt, dt_err_add=point_t_err)
    chi2 = residual_vector.T @ np.linalg.inv(point_pcov) @ residual_vector;
    return chi2



# def chi2_distance_to(track_param, track_cov, point_xyz,t):
#     """
#     track_param: (x0,y0,z0, vx,vy,vz, t)
#     """
#     _x,_y,_z= point_xyz;
#     x0,y0,z0, vx,vy,vz, t0 = track_param
#     dy = _y-y0
    
#     # Transform track covariance to covariance of (x,y,z)
#     jac=np.array([[     1, -vx/vy,    0,         dy/vy,    -vx*dy/(vy*vy),        0,        0],
#           [ 0, -vz/vy,  1,            0,    -vz*dy/(vy*vy),    dy/vy,        0],
#             [0,  -1/vy,     0,            0,       -dy/(vy*vy),        0,         1]])
#     CovMatrix_vertex = jac @ track_cov @ jac.transpose();
    
#     residual_vector = np.array([_x - (x0+vx*dy/vy),     _z- (z0+vz*dy/vy),        t- (t0+dy/vy)])
#     chi2 = residual_vector.transpose() @np.linalg.inv(CovMatrix_vertex)@residual_vector;
    
#     return chi2

# def chi2_distance_to_2(track_param, track_cov, point_xyz,t):
#     """
#     track_param: (x0,y0,z0, vx,vy,vz, t)
#     """
#     _x,_y,_z= point_xyz;
#     x0,y0,z0, vx,vy,vz, t0 = track_param
#     dt = t-t0
    
#     # Transform track covariance to covariance of (x,y,z)
#     jac=np.array([[     1, 0,    0,         dt,   0,        0,        -vx],
#                   [ 0, 1,  0,            0,    dt,    0,        -vy],
#                     [0, 0,     1,            0,       0,        dt,         -vz]])
#     CovMatrix_vertex = jac @ track_cov @ jac.transpose();
    
#     residual_vector = np.array([_x - (x0+vx*dt),     _y- (y0+vy*dt),    _z- (z0+vz*dt)])
#     chi2 = residual_vector.transpose() @np.linalg.inv(CovMatrix_vertex)@residual_vector;
    
#     return chi2

In [ ]:
def get_km_alltracks_old(filename, results_fit = None, tree_name="integral_tree", truth_pids = [13,13], nevents=-1):
    """
    Parameters
    ---
    nevents: 
      -1: all events
      (start, stop) from start to stop
    """
    
    if results_fit is None:
        results_fit={}

        
    results_fit["Entry"]=[]               # ROOT event entry number
    results_fit["ndigi"]=[]               # Total number of digitized hits
    results_fit["Digi_track_id"]=[]
    results_fit["mask_recon_success_track"]=[]    # Boolean mask 
    results_fit["mask_recon_success_vertex"]=[]   # Boolean mask 
    results_fit["mask_reconstructible_vertex"]=[]   # Boolean mask, if an event has two tracks with 4+ hits. The track is required to be from the direct input
    results_fit["mask_reconstructible2_vertex"]=[]   # Boolean mask, if an event has two tracks with 4+ hits. The track can be from any particle.  

    
    results_fit["tracks_truth"]=[]
    results_fit["tracks_truth_n"]=[]
    results_fit["tracks_truth_nlayer"]=[]
    results_fit["tracks_truth_pdgids"]=[]
    
    results_fit["tracks_recon"]=[]
    results_fit["tracks_recon_n"]=[]
    results_fit["tracks_recon_error"]=[]          # KF parameter uncertainty. Already taken sqrt()
    results_fit["tracks_ndigi"]=[]          #  number of digitized hits in the track
    results_fit["tracks_ndigi_false"]=[]    #  number of digitized hits in the track that are not from the truth
    results_fit["tracks_purity"]=[]         # List of PDG id of hits in this track 
    results_fit["tracks_pdgids"]=[]         # List of PDG id of hits in this track 
    results_fit["tracks_chi2"]=[]           # KF fit chi2
    
    results_fit["vertices_truth"]=[]
    results_fit["vertices_ntrack"]=[]
    results_fit["vertices_ntrack_truth"]=[]
    results_fit["vertices_recon"]=[]
    results_fit["vertices_recon_n"]=[]
    results_fit["vertices_recon_error"]=[]
    results_fit["vertices_recon_cov"]=[]
    results_fit["vertices_chi2"]=[]
    


    ev = event.Event(filename, 0, tree_name=tree_name)
    Tree=ev.Tree
    nevents_total = int(ev.Tree.GetEntries())
    cut=cutflow.sample_space("")
    
    if nevents==-1:
        nevents = [0, nevents_total]
    elif type(nevents) is int:
        if nevents_total<nevents:
            print(f"Requested events exceed total {nevents_total}")
        nevents = [0, min(nevents,nevents_total)]
    else:
        if nevents_total<nevents[1]:
            print(f"Requested events exceed total {nevents_total}")        
        nevents = [nevents[0], min(nevents[1],nevents_total)]

    for i_event in tqdm(range(nevents[0], nevents[1])):
        ev.EventNumber=i_event
        ev.Tree.GetEntry(i_event)
        ev.ExtractTruthPhysics_list()
        
        par_km_ndigi = ev.Tree.Digi_x.size()
        
        results_fit["Entry"].append(i_event)
        results_fit["ndigi"].append(par_km_ndigi)
        results_fit["Digi_track_id"].append(util.c2list(ev.Tree.Digi_track_id))
        ids=np.array(results_fit["Digi_track_id"][-1])
        ys = np.array(util.c2list(ev.Tree.Digi_y))
        n_reconstructible=0
        n_reconstructible2=0
        for g4id in range(0,200):
            if len(np.unique(ys[ids==g4id]))>=4:
                n_reconstructible+=1
        for g4id in np.unique(ids):
            if len(np.unique(ys[ids==g4id]))>=4:
                n_reconstructible2+=1                
        results_fit["vertices_ntrack_truth"].append(n_reconstructible)
        if n_reconstructible>=2:
            results_fit["mask_reconstructible_vertex"].append(True)
        else:
            results_fit["mask_reconstructible_vertex"].append(False)   
        if n_reconstructible2>=2:
            results_fit["mask_reconstructible2_vertex"].append(True)
        else:
            results_fit["mask_reconstructible2_vertex"].append(False)               

        # Get truth (speed need to be calculated by hand)
        try:
            # Truth position and speed
            TruthTracks = []
            Truths = []
            Truth_nlayer = []
            Truth_pdgids = []
            for track in ev.truthTrackList_list:
                if track[4][0] in truth_pids:
                    TruthTracks.append(track)
                    dt=track[3][1]-track[3][0]
                    vx=(track[0][1]-track[0][0])/dt
                    vy=(track[1][1]-track[1][0])/dt
                    vz=(track[2][1]-track[2][0])/dt
                    truth = [track[0][0], track[1][0], track[2][0], track[3][0],vx,vy,vz]  
                    Truths.append(truth)
                    Truth_nlayer.append(np.abs(cut.in_layer(track[1][-1])-cut.in_layer(track[1][0])))
                    Truth_pdgids.append(track[4][0])
            
            n_truthtracks=len(TruthTracks)
            results_fit["tracks_truth"].append(Truths)  
            results_fit["tracks_truth_n"].append(n_truthtracks)  
            results_fit["tracks_truth_nlayer"].append(Truth_nlayer)  
            
        except:
            n_truthtracks=0
            results_fit["tracks_truth"].append([[-9999]])
            results_fit["tracks_truth_n"].append(n_truthtracks)  
            results_fit["tracks_truth_nlayer"].append([[-9999]])
                  
        
        # If there is reconstruction:
        if len(ev.Tree.Track_k_m_z0)==0:
            tracks_recon = [[-9990, -9990, -9990, -9990, -9990, -9990, -9990]]
            tracks_recon_n = 0
            tracks_recon_error = [[-9990, -9990, -9990, -9990, -9990, -9990, -9990]]
            tracks_chi2 = [0]
            
            tracks_ndigi = [0]
            tracks_ndigi_false = [0]
            tracks_purity = [-999]
            tracks_pdgids = [0]
            results_fit["mask_recon_success_track"].append(False)
            
        else:
            tracks_recon = []
            tracks_recon_n = Tree.Track_k_m_z0.size()
            tracks_recon_error = []
            tracks_chi2 = []
            tracks_ndigi = []
            tracks_ndigi_false = []
            tracks_purity = []
            tracks_pdgids = []            
            
            
            # Select the reconstruction that is closest to truth
            track_digi_hit_inds = util.unzip(Tree.Track_k_m_hitIndices)
            track_truth_ids = util.unzip(Tree.Track_k_m_ids)
            track_digi_hit_len = np.array([len(i) for i in track_digi_hit_inds])
            if tracks_recon_n<n_truthtracks:
                results_fit["mask_recon_success_track"].append(False)
            else:
                results_fit["mask_recon_success_track"].append(True)                
                
            for i_track in range(len(TruthTracks)):
                truth = Truths[i_track]

                track_chi2s = []
                if len(track_digi_hit_inds)>1:
                    for track_ind in range(len(track_digi_hit_inds)):
                        recon_i = [Tree.Track_k_m_x0.at(track_ind), Tree.Track_k_m_y0.at(track_ind), Tree.Track_k_m_z0.at(track_ind), Tree.Track_k_m_t0.at(track_ind),Tree.Track_k_m_velX.at(track_ind), Tree.Track_k_m_velY.at(track_ind), Tree.Track_k_m_velZ.at(track_ind)]
                        recon_i_unc = [Tree.Track_k_m_ErrorX0.at(track_ind), Tree.Track_k_m_ErrorY0.at(track_ind), Tree.Track_k_m_ErrorZ0.at(track_ind), Tree.Track_k_m_ErrorT0.at(track_ind),Tree.Track_k_m_ErrorVx.at(track_ind), Tree.Track_k_m_ErrorVy.at(track_ind), Tree.Track_k_m_ErrorVz.at(track_ind)]
                        chi2 = util.chi2_calc(recon_i,truth,recon_i_unc)
                        track_chi2s.append(chi2)
                    track_ind = int(np.argmin(track_chi2s))
                else:
                    track_ind=0

                tracks_recon.append([ev.Tree.Track_k_m_x0.at(track_ind), ev.Tree.Track_k_m_y0.at(track_ind), ev.Tree.Track_k_m_z0.at(track_ind), ev.Tree.Track_k_m_t0.at(track_ind), ev.Tree.Track_k_m_velX.at(track_ind), ev.Tree.Track_k_m_velY.at(track_ind), ev.Tree.Track_k_m_velZ.at(track_ind)])
                tracks_recon_error.append([ev.Tree.Track_k_m_ErrorX0.at(track_ind), ev.Tree.Track_k_m_ErrorY0.at(track_ind), ev.Tree.Track_k_m_ErrorZ0.at(track_ind), ev.Tree.Track_k_m_ErrorT0.at(track_ind), ev.Tree.Track_k_m_ErrorVx.at(track_ind), ev.Tree.Track_k_m_ErrorVy.at(track_ind), ev.Tree.Track_k_m_ErrorVz.at(track_ind)])
                tracks_chi2.append(ev.Tree.Track_k_m_smooth_chi_sum.at(track_ind))
                
                track_hits_inds=track_digi_hit_inds[track_ind]
                truth_pid = TruthTracks[i_track][4][0]
                truth_track_id = TruthTracks[i_track][6][0]
                kalmantrack_truthtrack_ids = track_truth_ids[track_ind]
                
                tracks_pdgids.append([ev.Tree.Digi_pdg_id.at(i) for i in track_hits_inds])
                tracks_ndigi.append(len(track_hits_inds))
                tracks_ndigi_false.append(sum(np.array(kalmantrack_truthtrack_ids)!=truth_track_id))
                tracks_purity.append(1-tracks_ndigi_false[-1]/tracks_ndigi[-1])
                
        # Vertex========================================================
        if len(ev.Tree.Vertex_k_m_x)==0:
            vertex_recon = [[0]]
            vertex_recon_n=0
            vertex_recon_error=[[0]]
            vertices_chi2=[[0]]
            results_fit["mask_recon_success_vertex"].append(False) 
        else:
            results_fit["mask_recon_success_vertex"].append(True) 
            vertex_recon=[]
            vertex_recon_n = Tree.Vertex_k_m_x.size()
            vertex_recon_error=[]
            vertex_recon_cov=[]
            vertices_chi2=[]
            for iv in range(vertex_recon_n):
                v=[Tree.Vertex_k_m_x.at(iv),Tree.Vertex_k_m_y.at(iv),Tree.Vertex_k_m_z.at(iv),Tree.Vertex_k_m_t.at(iv)]
                vertex_recon.append(v)
                verr=[Tree.Vertex_k_m_ErrorX.at(iv),Tree.Vertex_k_m_ErrorY.at(iv),Tree.Vertex_k_m_ErrorZ.at(iv),Tree.Vertex_k_m_ErrorT.at(iv)]
                vertex_recon_error.append(verr)  
                vcov=[[Tree.Vertex_k_m_ErrorX.at(iv)**2, Tree.Vertex_k_m_cov_x_y.at(iv), Tree.Vertex_k_m_cov_x_z.at(iv), Tree.Vertex_k_m_cov_t_x.at(iv)],
                      [Tree.Vertex_k_m_cov_x_y.at(iv),   Tree.Vertex_k_m_ErrorY.at(iv)**2, Tree.Vertex_k_m_cov_y_z.at(iv), Tree.Vertex_k_m_cov_t_y.at(iv)],
                      [Tree.Vertex_k_m_cov_x_z.at(iv),   Tree.Vertex_k_m_cov_y_z.at(iv), Tree.Vertex_k_m_ErrorZ.at(iv)**2, Tree.Vertex_k_m_cov_t_z.at(iv)],
                      [Tree.Vertex_k_m_cov_t_x.at(iv),   Tree.Vertex_k_m_cov_t_y.at(iv), Tree.Vertex_k_m_cov_t_z.at(iv), Tree.Vertex_k_m_ErrorT.at(iv)**2]]
                vertex_recon_cov.append(vcov)
                vertices_chi2.append(Tree.vertex_k_m_chi2.at(iv))
                
        vertex_truth_cms = [Tree.GenParticle_y.at(1)*0.1, -Tree.GenParticle_z.at(1)*0.1 + 8547, Tree.GenParticle_x.at(1)*0.1]
        vertex_track_inds = util.unzip(Tree.Vertex_k_m_trackIndices)
        vertex_ntrack = [len(ii) for ii in vertex_track_inds]
        
        
        results_fit["tracks_recon"].append(tracks_recon)
        results_fit["tracks_recon_n"].append(tracks_recon_n)
        results_fit["tracks_recon_error"].append(np.sqrt(tracks_recon_error))
        results_fit["tracks_chi2"].append(tracks_chi2)
        results_fit["tracks_ndigi"].append(tracks_ndigi)
        results_fit["tracks_ndigi_false"].append(tracks_ndigi_false)
        results_fit["tracks_purity"].append(tracks_purity)
        results_fit["tracks_pdgids"].append(tracks_pdgids)
        
        results_fit["vertices_truth"].append(vertex_truth_cms)
        results_fit["vertices_ntrack"].append(vertex_ntrack)
        results_fit["vertices_recon"].append(vertex_recon)
        results_fit["vertices_recon_n"].append(vertex_recon_n)
        results_fit["vertices_recon_error"].append(vertex_recon_error)
        results_fit["vertices_recon_cov"].append(vertex_recon_cov)
        results_fit["vertices_chi2"].append(vertices_chi2)
        
        
    for key in ["tracks_truth_n","tracks_recon_n","mask_recon_success_track", "mask_recon_success_vertex","vertices_recon_n", "vertices_ntrack_truth"]:
        results_fit[key]=np.array(results_fit[key])
        
        
    return results_fit

In [ ]:
def read_raw_vertex(filename):
    """
    Return:
    [[x, y, z, px, py, pz],[x, y, z, px, py, pz],...]
    """
    vertices = []
    with open(filename, "r") as f:
        while True:
            line = f.readline()
            if not line:
                break
            if "#" in line:
                continue
            else:
                line = line.split()
                if len(line)>0 and line[0] == "n":
                    while True:
                        line2 = f.readline()
                        if "#" in line2:
                            continue                           
                        line2 = line2.split()
                        if len(line2)>0:
                            break
                    vertex = [float(line2[1]), float(line2[2]), float(line2[3]), float(line[6]), float(line[7]), float(line[8])] # x, y, z, px, py, pz
                    vertices.append(vertex)
                else:
                    continue
    return np.array(vertices)


def read_raw_vertex_weight(filename):
    """
    Return:
    [[x, y, z, px, py, pz],[x, y, z, px, py, pz],...]
    """
    vertices = []
    weights = []
    with open(filename, "r") as f:
        while True:
            line = f.readline()
            if not line:
                break
            if "#" in line:
                if "### weight" in line:
                    line = line.split()
                    weights.append([float(line[2]), float(line[3]), float(line[4])])
                else:
                    continue
            else:
                line = line.split()
                if len(line)>0 and line[0] == "n":
                    while True:
                        line2 = f.readline()
                        if "#" in line2:
                            continue                           
                        line2 = line2.split()
                        if len(line2)>0:
                            break
                    vertex = [float(line2[1]), float(line2[2]), float(line2[3]), float(line[6]), float(line[7]), float(line[8])] # x, y, z, px, py, pz
                    vertices.append(vertex)
                else:
                    continue
    return np.array(vertices), np.array(weights)


def read_raw_vertex_ntracks(filename):
    """
    Return:
    list of number of tracks in each vertex
    """
    ntracks = []
    with open(filename, "r") as f:
        while True:
            line = f.readline()
            if not line:
                break
            if "#" in line:
                continue
            else:
                line = line.split()
                if len(line)>0:
                    if line[0] == "n":
                        ntracks.append(0)                
                    else:
                        ntracks[-1]+=1
    return np.array(ntracks)

In [ ]:
def get_SMS_decay_types(mX):
    m_mu = 0.1057
    m_pi = 0.139    
    if float(mX)<2*m_mu:
        Decay_types = ["ee"]
    elif float(mX)<2*m_pi:
        Decay_types = ["mumu"]
    elif float(mX)<0.7:
        Decay_types = ["mumu","pipi"]
    elif float(mX)<0.98:
        Decay_types = ["mumu","hadrons0.7to0.98gev"]
    elif float(mX)<2:
        Decay_types = ["mumu","hadrons1to2gev"]
    elif float(mX)<3.65:
        Decay_types = ["gg","ss","mumu"]  
    elif float(mX)<3.8:
        Decay_types = ["gg","ss","mumu","tautau",]          
    elif float(mX)<5:
        Decay_types = ["gg","ss","mumu","tautau", "cc"]    
        
    return Decay_types